# Progettazione e Popolamento del Database SQLite

## Obiettivi

- **Definire uno schema dati** robusto e coerente per le recensioni auto.
- **Creare la tabella** nel database SQLite con i tipi di dato e i vincoli appropriati.
- **Caricare i dati puliti** dal file CSV in un DataFrame pandas.
- **Adattare le colonne** del DataFrame per la compatibilità SQL (es. rimozione spazi nei nomi).
- **Popolare la tabella** con i dati del DataFrame.
- **Verificare l’inserimento** e visualizzare alcuni record di esempio.


## Fasi operative

1. **Impostazione ambiente e connessione**
   - Import delle librerie necessarie (`pandas`, `sqlite3`, `os`)
   - Definizione del percorso del database e creazione della directory se necessario
   - Connessione al database SQLite

2. **Definizione dello schema**
   - Creazione della tabella `auto_reviews` con i seguenti campi:
     - `id` (chiave primaria, autoincrementale)
     - `Brand`, `Model` (testo, obbligatori)
     - Colonne qualitative (`Drive`, `Quality_of_interior`, ecc.)
     - `Price` (reale)
     - `Overall` (reale)

3. **Caricamento dati**
   - Lettura del file CSV pulito in un DataFrame pandas

4. **Adattamento colonne**
   - Rimozione degli spazi nei nomi delle colonne per compatibilità SQL
   - Eliminazione di colonne non necessarie (es. `Fascia_Prezzo`)

5. **Popolamento della tabella**
   - Inserimento dei dati del DataFrame nella tabella `auto_reviews` tramite `to_sql`

6. **Verifica e ispezione**
   - Verifica del numero di record inseriti
   - Visualizzazione di alcuni esempi di record

7. **Chiusura connessione**
   - Chiusura sicura della connessione al database

---

## Output atteso

- Database SQLite popolato e pronto per analisi o interrogazioni future.
- Tabella `auto_reviews` con dati coerenti e facilmente consultabili.
- Report sintetico a schermo con conferma delle operazioni e alcuni esempi di dati inseriti.

In [1]:
import pandas as pd
import sqlite3
import os

In [3]:
# Percorso del database
db_path = '/mnt/c/data/auto_reviews.db'

# Crea la directory se non esiste
os.makedirs(os.path.dirname(db_path), exist_ok=True)

# Connessione al database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Crea la tabella auto_reviews con le colonne corrette
cursor.execute('''
CREATE TABLE IF NOT EXISTS auto_reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Brand TEXT NOT NULL,
    Model TEXT NOT NULL,
    Drive TEXT,
    Quality_of_interior TEXT,
    Infotainment_system TEXT,
    Comfort TEXT,
    Performance TEXT,
    Handling TEXT,
    Practicality TEXT,
    Reliability TEXT,
    Safety TEXT,
    Quality_of_construction TEXT,
    Noise TEXT,
    Engine TEXT,
    Price REAL,
    Overall REAL,
    Fascia_Prezzo TEXT
)
''')

print("Tabella 'auto_reviews' creata con successo!")

Tabella 'auto_reviews' creata con successo!


In [4]:
## Caricamento csv pulito
df = pd.read_csv("info/car_review_cleaned.csv")

df.head()

,Unnamed: 0,Brand,Model,Drive,Quality of interior,Infotainment system,Comfort,Performance,Handling,Practicality,Reliability,Safety,Quality of construction,Noise,Engine,Price,Overall,Fascia_Prezzo
0,0,BMW,M8 Convertible,Positive,Positive,Medium,Not mentioned,Positive,Not mentioned,Negative,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Positive,178366.5,8.0,Lusso
1,1,BMW,M8,Positive,Positive,Medium,Negative,Positive,Positive,Negative,Not mentioned,Not mentioned,Positive,Not mentioned,Positive,170176.5,8.0,Lusso
2,2,BMW,M8 Gran Coupe,Not mentioned,Positive,Positive,Medium,Positive,Positive,Negative,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Positive,167251.5,6.0,Lusso
3,3,BMW,X6 M Competition,Positive,Positive,Medium,Positive,Positive,Positive,Medium,Not mentioned,Not mentioned,Positive,Negative,Positive,160512.3,8.0,Lusso
4,4,BMW,X5 M Competition,Positive,Positive,Medium,Positive,Positive,Positive,Positive,Not mentioned,Not mentioned,Positive,Negative,Positive,157341.6,8.0,Lusso


In [5]:
# Inserisci il DataFrame nel database
# Prima rinomino le colonne per compatibilità SQL (rimuovo spazi)
df_db = df.copy()
df_db.columns = df_db.columns.str.replace(' ', '_')

# Inserimento dati
df_db.to_sql('auto_reviews', conn, if_exists='replace', index=False)

print(f"Dati inseriti con successo! {len(df_db)} righe inserite.")

# Verifica inserimento
cursor.execute("SELECT COUNT(*) FROM auto_reviews")
count = cursor.fetchone()[0]
print(f"Numero di record nel database: {count}")

# Mostra alcuni record di esempio
cursor.execute("SELECT Brand, Model, Price, Overall FROM auto_reviews LIMIT 5")
sample_data = cursor.fetchall()
print("\nEsempi di dati inseriti:")
for row in sample_data:
    print(f"Brand: {row[0]}, Model: {row[1]}, Price: €{row[2]}, Overall: {row[3]}")

# Chiudi connessione
conn.close()
print(f"\nDatabase salvato in: {db_path}")

Dati inseriti con successo! 199 righe inserite.
Numero di record nel database: 199

Esempi di dati inseriti:
Brand: BMW, Model: M8 Convertible, Price: €178366.5, Overall: 8.0
Brand: BMW, Model: M8, Price: €170176.5, Overall: 8.0
Brand: BMW, Model: M8 Gran Coupe, Price: €167251.5, Overall: 6.0
Brand: BMW, Model: X6 M Competition, Price: €160512.3, Overall: 8.0
Brand: BMW, Model: X5 M Competition, Price: €157341.6, Overall: 8.0

Database salvato in: /mnt/c/data/auto_reviews.db


## Architettura della Soluzione – Database

### Scelte Progettuali

#### Database: **SQLite**
- **Motivazione:**  
  SQLite è un database relazionale leggero, serverless e portabile, ideale per progetti di analisi dati e prototipazione. Non richiede installazione di server aggiuntivi e consente una gestione semplice dei dati tramite file locale.
- **Vantaggi:**  
  - Facilità di integrazione con Python e pandas  
  - Nessuna configurazione server  
  - Ottimo per dataset di piccole/medie dimensioni  
  - File unico facilmente condivisibile

#### Connettori: **sqlite3** (standard Python)
- **Motivazione:**  
  Il modulo `sqlite3` è incluso nella libreria standard di Python, garantendo compatibilità e semplicità d’uso senza dipendenze esterne.
- **Integrazione:**  
  Utilizzato insieme a pandas per importare dati dal CSV e popolare la tabella tramite il metodo `to_sql`.

#### Pubblicazione
- **Modalità:**  
  Il database viene generato e salvato come file `.db` in una directory condivisa o versionata (es. `/mnt/c/data/auto_reviews.db`).
- **Vantaggi:**  
  - Facilità di distribuzione e backup  
  - Accessibile da qualsiasi ambiente Python  
  - Pronto per analisi o interrogazioni successive

---

**In sintesi:**  
La soluzione prevede l’estrazione e pulizia dei dati tramite pandas, la creazione e popolamento di una tabella relazionale in SQLite tramite `sqlite3`, e la pubblicazione del database come file locale pronto per analisi future. Questa architettura garantisce semplicità, portabilità e rapidità di sviluppo.